In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import time
%matplotlib inline
#import evolution
from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
vc = VirtualCoach(environment='local')

INFO: [2017-09-26 16:35:27,442 - rospy.topics] topicmanager initialized


INFO: [2017-09-26 16:35:27,677 - Configuration] Loading configuration file config.json
INFO: [2017-09-26 16:35:27,680 - Configuration] Using user specified environment: local
INFO: [2017-09-26 16:35:27,744 - VirtualCoach] Ready.


## Helper Functions
Some helper functions to calculate the fitness function, plot the robot's trajectory and the wheel speeds. These functions are specific to this experiment.

In [2]:
def fitness_function(wheel_speeds):
    left_wheel = ([float(t[1]) for t in wheel_speeds[2:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[2:]])
    fitness = 0
    for i in range(len(left_wheel)):
        if left_wheel[i] >= 0 and right_wheel[i] >= 0:
                   fitness += (left_wheel[i] + right_wheel[i])
    return fitness/float(2*len(left_wheel))

def get_top_performers(population, fitness_log, num_performers=15):
    """
    Extract the indices of the top individuals from the fitness log.
    
    :param fitness_log: fitness function scores for all individuals in a population
    :param num_performers: number for top performers to look for. Default value is
                           15, which corresponds to a truncation threshold of 25% in
                           this experiment.
    """
    top_performers = []
    for i in range(num_performers):
        max_index = np.argmax(fitness_log)
        print max_index
        top_performers.append(population[max_index])
        fitness_log[max_index] = -1

    return top_performers

def plot_trajectory(trajectory):
    plt.figure()
    plt.gca()
    plt.xticks([], [])
    plt.yticks([], [])
    plt.ylim(-3, 3)
    plt.xlim(-3.9, 3.9)
    x_axis = [x[0] for x in trajectory[2:]]
    y_axis = [y[1] for y in trajectory[2:]]
    plt.plot([float(x) for x in x_axis], [float(y) for y in y_axis])

def plot_wheel_speeds(wheel_speeds):
    left_wheel = ([float(t[1]) for t in wheel_speeds[1:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[1:]])
    plt.plot(range(len(left_wheel)), left_wheel, 'b')
    plt.plot(range(len(right_wheel)), right_wheel, 'r')

Evolutionary Algorithm helper functions

In [3]:
def one_point_crossover(parent1, parent2):
        parent1 = parent1.reshape(290)
        parent2 = parent2.reshape(290)
        child1 = np.zeros(290, dtype=int)
        child2 = np.zeros(290, dtype=int)
        point = np.random.randint(len(parent1))
        for i in range(point):
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        for i in range(point, 290):
            child1[i] = parent2[i]
            child2[i] = parent1[i]
        child1 = child1.reshape(10, 29)
        child2 = child2.reshape(10, 29)
        return child1, child2
    
def bit_mutation(population):
        for individual in population:
            individual = individual.reshape(290)
            for i in range(290):
                if np.random.rand() < 0.05:
                    individual[i] = 0 if individual[i] else 1
        return population
    
def get_unique_pairs(population):
    pairs = []
    for i in range(len(population)):
        for j in range(i+1, len(population)):
            pairs.append((i, j))
    return pairs
            
def evolve_new_generation(top_performers):
    population = []
    for i in range(len(top_performers)):
        for j in range(4):
            population.append(top_performers[i])
    pairs = get_unique_pairs(population)
    for i in pairs:
        if np.random.rand() < 0.1:
            parent1 = population[i[0]]
            parent2 = population[i[1] - 1]
            child1, child2 = one_point_crossover(parent1, parent2)
            population[i[0]] = child1
            population[i[1]] = child2

    population = bit_mutation(population)
    rand = np.random.randint(len(population))
    population[rand] = top_performers[0]
    return population

## The Brain
The PyNN script that creates the neural network stored as a string. A new binary genetic string that encodes the connections between neurons is passed on each run.

In [4]:
brain = """from hbp_nrp_cle.brainsim import simulator as sim
import numpy as np
import logging

logger = logging.getLogger(__name__)

def create_brain():

    dna = np.array([int(x) for x in '%s'.split(',')]).reshape(10, 29)

    BRAINPARAMS = {'v_rest': -60.0,
                    'cm': 4.0,
                    'tau_m': 4.0,
                    'tau_refrac': 2.0,
                    'tau_syn_E': 10.0,
                    'tau_syn_I': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -12.5,
                    'v_reset': -60.0}

    SENSORPARAMS = {'v_rest': -60.5,
                    'cm': 0.6,
                    'tau_m': 4.0,
                    'tau_refrac': 2.0,
                    'tau_syn_E': 10.0,
                    'tau_syn_I': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -60.0,
                    'v_reset': -60.5}

    SYNAPSE_PARAMS = {"weight": 1.0,
                      "delay": 2.0}

    population = sim.Population(28, sim.IF_cond_alpha())
    population[0:18].set(**SENSORPARAMS)
    population[18:28].set(**BRAINPARAMS)


    # Connect neurons
    CIRCUIT = population

    SYN = sim.StaticSynapse(**SYNAPSE_PARAMS)

    row_counter=0
    for row in dna:
    	logger.info(row)
        n = np.array(row)
        r_type = 'excitatory'
        for i in range(1,19):
            if n[i]==1:
            	logger.info(str(i-1)+' '+str(18+row_counter)+' '+r_type)
                sim.Projection(presynaptic_population=CIRCUIT[i-1:i], postsynaptic_population=CIRCUIT[18+row_counter:19+row_counter], connector=sim.OneToOneConnector(), synapse_type=SYN, receptor_type=r_type)
        if n[0]==0:
            r_type = 'inhibitory'
        for i in range(19,29):
            if n[i]==1:
            	logger.info(str(18+row_counter)+' '+str(i-1)+' '+r_type)

                sim.Projection(presynaptic_population=CIRCUIT[18+row_counter:19+row_counter], postsynaptic_population=CIRCUIT[i-1:i], connector=sim.OneToOneConnector(), synapse_type=SYN, receptor_type=r_type)
        
        row_counter+=1

    sim.initialize(population, v=population.get('v_rest'))

    logger.debug("Circuit description: " + str(population.describe()))

    return population


circuit = create_brain()
"""

In [5]:
display_trial_tf = """@nrp.Robot2Neuron()
def display_trial_number(t):
    clientLogger.advertise('%s')
"""

## Run Experiment

In [6]:
class FloreanoExperiment(object):
    
    def __init__(self, population, generations):
        self.last_status = [None]
        self.population = population
        self.fitness_log = []
        self.sim = None
        self.started = False
        self.generations = generations
        self.sim_data = []

    def wait_condition(self, timeout, condition):
        start = time.time()
        while time.time() < start + timeout:
            time.sleep(0.25)
            if condition(self.last_status[0]):
                return
        raise Exception('Condition check failed')
        
    def on_status(self, status):
        self.last_status[0] = status
                 
    def save_simulation_data(self, trial):
        self.sim_data[trial].append([])
        wheel_speeds = self.sim.get_csv_data('wheel_speeds.csv')
        left_wheel = ([float(t[1]) for t in wheel_speeds[2:]])
        right_wheel = ([float(t[2]) for t in wheel_speeds[2:]])
        trajectory = self.sim.get_csv_data('robot_position.csv')
        fitness = fitness_function(wheel_speeds)
        self.sim_data[trial][-1] = {
            'fitness': fitness,
            'wheel_speeds': wheel_speeds,
            'left_wheel': left_wheel,
            'right_wheel': right_wheel,
            'trajectory': trajectory
        }

    def run_experiment(self):
        try:
            self.sim = vc.launch_experiment('floreano')
        except:
            time.sleep(1)
        self.sim.register_status_callback(self.on_status)
        for i in range(self.generations):
            self.sim_data.append([])
            for j in range(len(self.population)):
                genetic_string = ','.join(str(x) for x in population[j].ravel())
                self.sim.edit_brain(brain % genetic_string)
                self.sim.add_transfer_function(display_trial_tf % "Generation {}, Population {}".format(i, j) )
                self.sim.start()
                # run simulation for 40 seconds
                self.wait_condition(1000, lambda x: x['simulationTime'] == 40)
                self.sim.pause()
                self.save_simulation_data(i)
                self.sim.reset('full')
                self.wait_condition(100, lambda x: x['state'] == 'paused' and x['simulationTime'] == 0)
            self.fitness_log = [result['fitness'] for result in floreano_experiment.sim_data[i]]
            self.top_performers = get_top_performers(self.population, list(self.fitness_log))
            self.population = evolve_new_generation(self.top_performers)

In [ ]:
population = np.random.randint(2, size=(60, 10, 29)) # random population of 30 binary genetic strings
floreano_experiment = FloreanoExperiment(population, 16)
floreano_experiment.run_experiment()

INFO: [2017-09-26 16:36:04,255 - VirtualCoach] Preparing to launch floreano.
INFO: [2017-09-26 16:36:04,257 - VirtualCoach] Retrieving list of experiments.
INFO: [2017-09-26 16:36:04,287 - Simulation] Attempting to launch floreano on localhost.
INFO: [2017-09-26 16:36:14,536 - Simulation (floreano - localhost #1)] Simulation Successfully Created.
INFO: [2017-09-26 16:36:14,561 - Simulation (floreano - localhost #1)] Ready.
INFO: [2017-09-26 16:36:14,562 - Simulation (floreano - localhost #1)] Status callback registered.
INFO: [2017-09-26 16:36:14,563 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:36:14,572 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 16:36:14,573 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:36:14,580 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:36:14,592 - Simulation (floreano - localhost #1)] Attempti

INFO: [2017-09-26 16:37:51,968 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 16:37:51,970 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 16:37:52,215 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
INFO: [2017-09-26 16:37:52,659 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-09-26 16:37:53,160 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:37:53,167 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 16:37:53,167 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:37:53,172 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:37:53,177 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO

INFO: [2017-09-26 16:39:29,175 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 16:39:29,178 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 16:39:29,420 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
INFO: [2017-09-26 16:39:29,907 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-09-26 16:39:30,909 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:39:30,917 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 16:39:30,918 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:39:30,932 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:39:30,939 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO

INFO: [2017-09-26 16:41:08,399 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 16:41:08,400 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 16:41:08,629 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
INFO: [2017-09-26 16:41:09,124 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-09-26 16:41:09,626 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:41:09,632 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 16:41:09,633 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:41:09,637 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:41:09,642 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO

INFO: [2017-09-26 16:42:48,408 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 16:42:48,408 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 16:42:48,636 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
INFO: [2017-09-26 16:42:49,164 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-09-26 16:42:49,666 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:42:49,671 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 16:42:49,672 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:42:49,677 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 16:42:49,681 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO

...

INFO: [2017-09-26 17:54:29,540 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 17:54:29,543 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 17:54:29,753 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
INFO: [2017-09-26 17:54:30,219 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-09-26 17:54:30,470 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:54:30,477 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 17:54:30,478 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:54:30,486 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:54:30,493 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO

INFO: [2017-09-26 17:56:13,617 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 17:56:13,617 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 17:56:13,845 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
INFO: [2017-09-26 17:56:14,344 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-09-26 17:56:14,595 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:56:14,601 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 17:56:14,602 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:56:14,607 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:56:14,613 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO

INFO: [2017-09-26 17:57:52,219 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 17:57:52,220 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 17:57:52,443 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
INFO: [2017-09-26 17:57:52,902 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-09-26 17:57:53,404 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:57:53,409 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 17:57:53,413 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:57:53,420 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:57:53,424 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO

INFO: [2017-09-26 17:59:35,279 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 17:59:35,281 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 17:59:35,506 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
INFO: [2017-09-26 17:59:36,025 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-09-26 17:59:36,277 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:59:36,281 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 17:59:36,282 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:59:36,289 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 17:59:36,294 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO

INFO: [2017-09-26 18:01:15,503 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 18:01:15,507 - Simulation (floreano - localhost #1)] [Resetting the simulation] Loading: plane
INFO: [2017-09-26 18:01:15,735 - Simulation (floreano - localhost #1)] [Resetting the simulation] Restoring the brain
INFO: [2017-09-26 18:01:16,249 - Simulation (floreano - localhost #1)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-09-26 18:01:16,750 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 18:01:16,755 - Simulation (floreano - localhost #1)] Attempting to set Brain
INFO: [2017-09-26 18:01:16,756 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 18:01:16,762 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO: [2017-09-26 18:01:16,769 - Simulation (floreano - localhost #1)] Attempting to retrieve brain
INFO

In [ ]:
fig, axes = plt.subplots(len(floreano_experiment.sim_data), 2)
for i in range(len(floreano_experiment.sim_data)):
    axes[i, 0].set_ylim(-3, 3)
    axes[i, 0].set_xlim(-3.9, 3.9)
    axes[i, 0].set_xticks([], [])
    axes[i, 0].set_yticks([], [])
    x_axis = [x[0] for x in floreano_experiment.sim_data[i]['trajectory'][2:]]
    y_axis = [y[1] for y in floreano_experiment.sim_data[i]['trajectory'][2:]]
    axes[i, 0].plot([float(x) for x in x_axis], [float(y) for y in y_axis])

    left_wheel = ([float(t[1]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    right_wheel = ([float(t[2]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    axes[i, 1].plot(range(len(left_wheel)), left_wheel, 'b', label='Left Wheel')
    axes[i, 1].plot(range(len(right_wheel)), right_wheel, 'r', label='Right Wheel')
    axes[i, 1].set_ylim(-5, 5)
    axes[i, 1].set_xlabel('Time [ms]')
    axes[i, 1].set_ylabel('Speed m/s')

axes[0, 0].set_title('Robot Trajectory')
axes[0, 1].set_title('Wheel Speeds')
fig.set_figheight(25)
fig.set_figwidth(10)
